### Assignment 1: due week 5 just before class at 1759 hours
Given a fully connected Neural Network as follows:
1. Input (x1,x2): 2 nodes

2. First hidden layer: 10 nodes, with weights (w) and bias (b), sigmoid activation function

3. Second hidden layer: 10 nodes, with weights (w) and bias (b), sigmoid activation function

4. Output (predict): 1 node



(1) Implement this neural network in pytorch

(2) Generate the input data (x1,x2) \in [0,1] drawn from a uniform random distribution

(3) Generate the labels y = (x1*x1+x2*x2)/2

(4) Implement a loss function L = (predict-y)^2

(5) Use batch size of 1, that means feed data one point at a time into network and compute the loss. Do one time forward propagation with one data point.

(6) Compute the gradients using pytorch autograd:

a. dL/dw, dL/db

b. Print these values into a text file: torch_autograd.dat

In [169]:
import torch
torch.set_printoptions(precision=4)
#torch.__version__ 1.7.1
import numpy as np

N, D_in, H1, H2, D_out = 1, 2, 10, 10, 1

'''N is batch size, set N as 1 is what Q5 imply'''

torch.manual_seed(0)
X = torch.rand(N,2)
y = torch.empty(N,1)
y[0,0] = (X[0][0]**2 + X[0][1]**2)/2
print("Input X: ",X)
print("Input y: ",y)

class TwoLayerNet(torch.nn.Module):
    
    def __init__(self, D_in, H1,H2, D_out):
        
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, H2)
        self.linear3 = torch.nn.Linear(H2, D_out)

    def forward(self, x):
        sigmoid = torch.nn.Sigmoid()        
        h1_sigmoid = sigmoid(self.linear1(x))       
        h2_sigmoid = sigmoid(self.linear2(h1_sigmoid))  
        y_pred = self.linear3(h2_sigmoid)
        return y_pred

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H1,H2, D_out)

#Save the initial parameters for my own defined BP process
b3 = model.linear3.bias
w3 = model.linear3.weight.t()
b2 = model.linear2.bias
w2 = model.linear2.weight.t()
b1 = model.linear1.bias
w1 = model.linear1.weight.t()

loss_fn = torch.nn.MSELoss(reduction='sum')
#def loss_fn(y, y_pred):
#    return torch.sum((y_pred - y) ** 2) #pretty the same with MSELoss

learning_rate = 0.1

for t in range(1):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(X)
    
    # Compute and print loss
    loss = loss_fn(y_pred, y)
    print('step %d: loss=%f   ' % (t+1, loss.item()))   
    model.zero_grad()
    loss.backward()
    
    #As we only need the 1st time gradient, no need to update the parameters
    '''
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
    '''  
    
    # in Linear class __init__，the shape of weight[out_features, in_features], need to be transformed
    print("First hidden layer weights gradient: \n",model.linear1.weight.grad.t())
    print("First hidden layer bias gradient: \n", model.linear1.bias.grad)
    print("Second hidden layer weight gradient:\n", model.linear2.weight.grad.t())
    print("Second hidden layer bias gradient: \n",model.linear2.bias.grad)
    print("Output layer bias gradient: \n",model.linear3.bias.grad)
    print("Output layer weight gradient: \n",model.linear3.weight.grad.t())
    
print("Predicted y:",y_pred)

Input X:  tensor([[0.4963, 0.7682]])
Input y:  tensor([[0.4182]])
step 1: loss=0.986679   
First hidden layer weights gradient: 
 tensor([[-0.0028,  0.0071, -0.0034, -0.0120, -0.0009, -0.0063, -0.0078, -0.0040,
          0.0051,  0.0050],
        [-0.0043,  0.0110, -0.0053, -0.0185, -0.0015, -0.0097, -0.0121, -0.0062,
          0.0080,  0.0078]])
First hidden layer bias gradient: 
 tensor([-0.0056,  0.0143, -0.0069, -0.0241, -0.0019, -0.0126, -0.0158, -0.0081,
         0.0104,  0.0102])
Second hidden layer weight gradient:
 tensor([[-0.0402,  0.0034,  0.0420,  0.0224, -0.0303,  0.0003,  0.0236,  0.0363,
          0.0421,  0.0394],
        [-0.0839,  0.0072,  0.0875,  0.0468, -0.0631,  0.0006,  0.0492,  0.0757,
          0.0879,  0.0823],
        [-0.0766,  0.0066,  0.0799,  0.0427, -0.0577,  0.0006,  0.0449,  0.0691,
          0.0803,  0.0751],
        [-0.0726,  0.0062,  0.0758,  0.0405, -0.0546,  0.0006,  0.0426,  0.0655,
          0.0761,  0.0712],
        [-0.0804,  0.0069,  0.0839


(7) Implement the forward propagation and backpropagation algorithm from scratch,without using pytorch autograd, compute the gradients using your implementation

a. dL/dw, dL/db

b. Print these values into a text file: my_autograd.dat

(8) Compare the two files torch_autograd.dat and my_autograd.dat and show that they give the same values up to numerical precision errors

In [170]:
def sigmoid_derivationx(y):
    return y * (1 - y)

learning_rate = 0.1
for t in range(1):
    # Forward pass: compute predicted y 
    h1 = X.mm(w1) +b1 # N*D_in x D_in* H1 + N*H1 = N*H1
    h1_sigmoid = torch.sigmoid(h1)   #N*H1
    h2 = h1_sigmoid.mm(w2) +b2   #N*H1 x H1*H2 + N*H2  = N *H2
    h2_sigmoid = torch.sigmoid(h2)#N * H2   
    y_pred  = h2_sigmoid.mm(w3) +b3 #N*H2 x H2*D_out + N*D_out= N * D_out
         
    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    print('step %d: loss=%f   ' % (t+1, loss))
    
    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = -2.0 *(y - y_pred)#N    
    delta = grad_y_pred#N*D_out                           
    grad_w3 = h2_sigmoid.T.mm(delta)#H2*N mm N*D_out =    H2* D_out                    
    grad_b3 = delta  #N*D_out 
    
    delta = delta.mm(w3.T)*sigmoid_derivationx(h2_sigmoid)#N*D_out x D_out*H2 = N*10 . N *10 =N *10
    grad_w2 = h1_sigmoid.T.mm(delta)#10*N  x N*10 = 10*10
    grad_b2 = delta   # 1*N x N*10  = 1*10
    
    delta = delta.mm(w2.T)*sigmoid_derivationx(h1_sigmoid)#N*10 X 10*10 = N *10 * N *10 = N *10
    grad_w1 = X.T.mm(delta)# 2*N X N*10 = 2*10
    grad_b1 =  delta  # 1*N x N * 10 
     
    print("First hidden layer weights gradient: \n",grad_w1)
    print("First hidden layer bias gradient: \n", grad_b1)
    print("Second hidden layer weight gradient:\n", grad_w2)
    print("Second hidden layer bias gradient: \n",grad_b2)
    print("Output layer bias gradient: \n",grad_b3)   
    print("Output layer weight gradient: \n",grad_w3)  
    
    #update weights
    w3 = w3 - learning_rate * grad_w3  
    w2 = w2 - learning_rate * grad_w2
    w1 = w1 - learning_rate * grad_w1
    
    b3 = b3 - learning_rate * grad_b3
    b2 = b2 - learning_rate * grad_b2
    b1 = b1 - learning_rate * grad_b1  
print(y_pred)

step 1: loss=0.986679   
First hidden layer weights gradient: 
 tensor([[-0.0028,  0.0071, -0.0034, -0.0120, -0.0009, -0.0063, -0.0078, -0.0040,
          0.0051,  0.0050],
        [-0.0043,  0.0110, -0.0053, -0.0185, -0.0015, -0.0097, -0.0121, -0.0062,
          0.0080,  0.0078]], grad_fn=<MmBackward>)
First hidden layer bias gradient: 
 tensor([[-0.0056,  0.0143, -0.0069, -0.0241, -0.0019, -0.0126, -0.0158, -0.0081,
          0.0104,  0.0102]], grad_fn=<MulBackward0>)
Second hidden layer weight gradient:
 tensor([[-0.0402,  0.0034,  0.0420,  0.0224, -0.0303,  0.0003,  0.0236,  0.0363,
          0.0421,  0.0394],
        [-0.0839,  0.0072,  0.0875,  0.0468, -0.0631,  0.0006,  0.0492,  0.0757,
          0.0879,  0.0823],
        [-0.0766,  0.0066,  0.0799,  0.0427, -0.0577,  0.0006,  0.0449,  0.0691,
          0.0803,  0.0751],
        [-0.0726,  0.0062,  0.0758,  0.0405, -0.0546,  0.0006,  0.0426,  0.0655,
          0.0761,  0.0712],
        [-0.0804,  0.0069,  0.0839,  0.0448, -0.060